# C10-COVID-19 Characterization on Lung Radiographs

For Coding.Waterkant hackathon

In [0]:
from enum import Enum
import json
import os
import datetime

import cv2
import matplotlib.pyplot as plt
import numpy as np
from imutils import paths

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, DirectoryIterator
from tensorflow.keras.applications import VGG16, InceptionV3
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense, Input, Model
from tensorflow.keras.optimizers import Adam, SGD

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Defining constants

In [0]:
ROOT_DIR = './drive/My Drive/Waterkant/'
TRAIN_IMAGES_PATH = os.path.join(ROOT_DIR, 'chest_xray/train')
VAL_IMAGES_PATH = os.path.join(ROOT_DIR, 'chest_xray/val')
TEST_IMAGES_PATH = os.path.join(ROOT_DIR, 'chest_xray/test')
FINAL_TEST_IMAGES_PATH = os.path.join(ROOT_DIR, 'chest_xray/final')

In [0]:
INIT_LR = 1e-3
EPOCHS = 25
BATCH_SIZE = 32

In [0]:
class ModelChoices(Enum):
  VGG = 1
  INCEPTIONV3 = 2
  XCEPTION = 3
  ENSEMBLE = 0

In [0]:
selected_model = ModelChoices.VGG
# Note: InceptionV3 model slightly better than VGG on pre-COVID dataset,
# whereas VGG performed significantly better on pre-COVID dataset
# Xception and ensemble model couldn't be tested due to training time constraints

### Load datasets

In [0]:
resize_to = (224, 224)  # For use with pre-trained models

In [0]:
train_aug = ImageDataGenerator(brightness_range=[0.2, 1.0], zoom_range=0.2, fill_mode="nearest")
train_set = DirectoryIterator(TRAIN_IMAGES_PATH, train_aug, target_size=resize_to, color_mode='rgb', batch_size=BATCH_SIZE, class_mode='binary')

Found 5216 images belonging to 2 classes.


In [0]:
val_aug = ImageDataGenerator(rotation_range=15, fill_mode="nearest")
validation_set = DirectoryIterator(VAL_IMAGES_PATH, val_aug, target_size=resize_to, color_mode='rgb', batch_size=BATCH_SIZE, class_mode='binary')

Found 16 images belonging to 2 classes.


In [0]:
test_set = DirectoryIterator(TEST_IMAGES_PATH, None, target_size=resize_to, color_mode='rgb', batch_size=BATCH_SIZE, class_mode='binary')

Found 16 images belonging to 2 classes.


### Instantiate preferred model

In [0]:
if selected_model == ModelChoices.VGG:
	# load a pre-trained VGG16 network, ensuring the head FC layer sets are left off
	base_model = VGG16(weights="imagenet", include_top=False,
		input_tensor=Input(shape=(*resize_to, 3)))

	n_output_classes = 2

	# construct the head of the model that will be placed on top of the
	# the base model
	headModel = base_model.output
	headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
	headModel = Flatten(name="flatten")(headModel)
	headModel = Dense(64, activation="relu")(headModel)
	headModel = Dropout(0.5)(headModel)
	headModel = Dense(n_output_classes, activation="softmax")(headModel)
	# place the head FC model on top of the base model (this will become
	# the actual model we will train)
	model = Model(inputs=base_model.input, outputs=headModel)
	# loop over all layers in the base model and freeze them so they will
	# *not* be updated during the first training process
	for layer in base_model.layers:
		layer.trainable = False

	# compile our model
	print("[INFO] compiling model...")
	opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
	model.compile(optimizer=opt, loss="binary_crossentropy",
		metrics=["accuracy"])

58892288/58889256 [==============================] - 0s 0us/step
[INFO] compiling model...


In [0]:
if selected_model == ModelChoices.INCEPTIONV3:
  # create the base pre-trained model
  base_model = InceptionV3(weights='imagenet', include_top=False)

  n_output_classes = 2

  # add a global spatial average pooling layer
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  # let's add a fully-connected layer
  x = Dense(1024, activation='relu')(x)
  # and a logistic layer -- let's say we have 200 classes
  predictions = Dense(n_output_classes, activation='softmax')(x)

  # this is the model we will train
  model = Model(inputs=base_model.input, outputs=predictions)

  # first: train only the top layers (which were randomly initialized)
  # i.e. freeze all convolutional InceptionV3 layers
  for layer in base_model.layers:
      layer.trainable = False

  # compile the model (should be done *after* setting layers to non-trainable)
  print("[INFO] compiling model...")
  opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
  model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])

  # train the model on the new data for a few epochs
  model.fit(train_set, 
      steps_per_epoch=len(train_set),
      validation_data=validation_set,
      validation_steps=len(validation_set),
      epochs=3
  )

  # at this point, the top layers are well trained and we can start fine-tuning
  # convolutional layers from inception V3. We will freeze the bottom N layers
  # and train the remaining top layers.

  # let's visualize layer names and layer indices to see how many layers
  # we should freeze:
  for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

  # we chose to train the top 2 inception blocks, i.e. we will freeze
  # the first 249 layers and unfreeze the rest:
  for layer in model.layers[:249]:
    layer.trainable = False
  for layer in model.layers[249:]:
    layer.trainable = True

  # we need to recompile the model for these modifications to take effect
  # we use SGD with a low learning rate
  model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')


In [0]:
if selected_model == ModelChoices.XCEPTION:
  base_model = tf.keras.applications.Xception(input_shape=(*resize_to, 3), include_top=False, weights="imagenet")

  for layer in base_model.layers:
    layer.trainable = False

  n_output_classes = 2

  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
  flatten_layer = Flatten(name="flatten")(global_average_layer)
  dense_layer = Dense(64, activation="relu")(flatten_layer)
  dropout_layer = Dropout(0.5)(dense_layer)
  prediction_layer = tf.keras.layers.Dense(n_output_classes, activation='softmax')(dropout_layer)

  model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)
  model.compile(optimizer=tf.keras.optimizers.Adam(lr=INIT_LR), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=["accuracy"])


In [0]:
if selected_model == ModelChoices.ENSEMBLE:
  def load_all_models():
      all_models = []
      model_names = [
        'pretrained_on_vgg_16.model',
        'pretrained_on_inceptionv3.model'
      ]
      for model_name in model_names:
          filename = os.path.join(ROOT_DIR, 'models', model_name)
          model = tf.keras.models.load_model(filename)
          all_models.append(model)
          print('loaded:', filename)
      return all_models

  models = load_all_models()
  for i, model in enumerate(models):
      for layer in model.layers:
          layer.trainable = False

  ensemble_visible = [model.input for model in models]
  ensemble_outputs = [model.output for model in models]

  merge = tf.keras.layers.concatenate(ensemble_outputs)
  merge = tf.keras.layers.Dense(10, activation='relu', name="DENSE_SEMI_FINAL")(merge)
  output = tf.keras.layers.Dense(2, activation='sigmoid', name="DENSE_FINAL")(merge)
  model = tf.keras.models.Model(inputs=ensemble_visible, outputs=output)

  # slightly larger learning rate for ensemble model
  model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=["accuracy"])

loaded: ./drive/My Drive/Waterkant/models/pretrained_on_vgg_16.model


### Training 🏃

In [0]:
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

log_dir = os.path.join(ROOT_DIR, "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [0]:
%tensorboard --logdir drive/My\ Drive/Waterkant/logs/fit

In [0]:
H = model.fit(train_set, 
    steps_per_epoch=len(train_set),
	  validation_data=validation_set,
	  validation_steps=len(validation_set),
	  epochs=EPOCHS,
    callbacks=[early_stop_callback, tensorboard_callback]
)

### Saving model to disk

In [0]:
resize_str = str(resize_to).replace('(', '').replace(')', '').replace(', ', '_')
MODEL_NAME = f"c10-covid19-characterization_{selected_model.name}_{resize_str}"
MODEL_NAME

'c10-covid19-characterization_VGG_224_224'

In [0]:
# serialize the model to disk
print("[INFO] saving COVID-19 detector model...")
model.save(f"{MODEL_NAME}.model", save_format="h5")

[INFO] saving COVID-19 detector model...


### Loading already trained model from disk

In [0]:
### For loading already trained model
model_name = 'pretrained_on_vgg_16.model'
filename = os.path.join(ROOT_DIR, 'models', model_name)

model = tf.keras.models.load_model(filename)

### Running against test

In [0]:
def get_processed_image(image_path):
  # load the image, swap color channels, and resize it to be a fixed
  # 224x224 pixels while ignoring aspect ratio
  image = cv2.imread(image_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image, resize_to)

  return image

# we had to load the test dataset in a different way, as the labels for them weren't present
final_test_images_paths = list(paths.list_images(FINAL_TEST_IMAGES_PATH))
test_set = np.array([get_processed_image(test_images_path) for test_images_path in final_test_images_paths])

In [0]:
print("[INFO] evaluating network on test set...")
predIdxs = model.predict(test_set, batch_size=BATCH_SIZE)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

[INFO] evaluating network...


In [0]:
predIdxs

array([1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1])

### Format predictions for submission

In [0]:
def format_predictions(predIdxs, images_path):
  all_predictions = []
  for i in range(len(test_set)):
    output = dict({
        'image': os.path.basename(images_path[i]), 
        'prediction': int(predIdxs[i])
      })
    all_predictions.append(output)
  return sorted(all_predictions, key=lambda k: k['image']) 

In [0]:
predictions = format_predictions(predIdxs, final_test_images_paths)

print(json.dumps(predictions))

with open(f"{MODEL_NAME}_final.json", 'w') as F:
  # Use the json dumps method to write the list to disk
  F.write(json.dumps(predictions))  

[{"image": "700.jpg", "prediction": 1}, {"image": "701.jpg", "prediction": 1}, {"image": "702.jpg", "prediction": 1}, {"image": "703.jpg", "prediction": 1}, {"image": "704.jpg", "prediction": 0}, {"image": "705.jpg", "prediction": 0}, {"image": "706.jpg", "prediction": 1}, {"image": "707.jpg", "prediction": 0}, {"image": "708.jpg", "prediction": 1}, {"image": "709.jpg", "prediction": 1}, {"image": "710.jpg", "prediction": 1}, {"image": "711.jpg", "prediction": 0}, {"image": "712.jpg", "prediction": 0}, {"image": "713.jpg", "prediction": 1}, {"image": "714.jpg", "prediction": 0}, {"image": "715.jpg", "prediction": 0}, {"image": "716.jpg", "prediction": 0}, {"image": "717.jpg", "prediction": 1}, {"image": "718.jpg", "prediction": 1}, {"image": "719.jpg", "prediction": 1}, {"image": "720.jpg", "prediction": 1}, {"image": "721.jpg", "prediction": 0}, {"image": "722.jpg", "prediction": 1}, {"image": "723.jpg", "prediction": 1}, {"image": "724.jpg", "prediction": 0}, {"image": "725.jpg", "pr